#AUTOGRAD: 자동 미분

PyTorch의 모든 신경망의 중심에는 autograd 패키지가 있다. autograd 패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공한다. 순전파(forward) 단계에서는 수행하는 모든 연산을 기억하고, 역전파(backward)에서는 연산들을 replay한다. 

<br/>

**tensor를 생성하고 requires_grad=True를 설정하여 연산을 기록합니다.**

In [1]:
import torch

x = torch.ones(2,2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


<br/>

**tensor에 연산을 수행합니다.**

In [2]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


<br/>

**y는 연산의 결과로 생성된 것이므로 grad_fn을 갖습니다. grad_fn은 미분값을 계산하는 함수에 대한 정보를 말합니다.** 

In [3]:
print(y.grad_fn)
print(y.grad)

None


<br/>

**y에 다른 연산을 수행합니다.**

In [4]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


<br/>

**.require_grad_(...)는 기존 Tensor의 requires_grad값을 바꿔치기(in-place)하여 변경합니다. 입력값이 지정되지 않으면 기본값은 False입니다.**

In [5]:
a = torch.randn(2,2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


<br/><br/><br/>

## 변화도(Gradient)

<br/><br/>
이제 역전파(backprop)을 해보겠습니다. out은 하나의 스칼라 값만 가지고 있기 때문에, out.backward()는 out.backward(torch.tensor(1.))과 동일합니다.

In [0]:
out.backward()

<br/>

변화도 d(out)/dx를 출력합니다.

In [7]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


<br/>

**grad는 data가 거쳐온 layer에 대한 미분값이 축적되는 것을 말합니다.**

<br/>

이처럼 torch.autograd는 벡터-Jacobian 곱을 계싼하는 엔진입니다. 

In [9]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:  #data.norm은 norm2를 의미
  y = y * 2

print(y)

tensor([ 355.6519, 1157.0955,  221.1868], grad_fn=<MulBackward0>)


<br/>

tensor가 스칼라이면, backward()의 인자를 기록할 필요가 없습니다. 그러나 여러개의 값들을 가진경우, 일치하는 형태의 텐서 인자를 열거해야 합니다. 이 경우 y는 더 이상 스칼라 값이 아닙니다. torch.autograd는 전체 야코비안을 직접 계산할 수는 없지만, 벡터-야코비안 곱은 간단히 backward에 해당 벡터를 인자로 제공하여 얻을 수 있습니다.

In [10]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


<br/>

또한 with torch.no_grad(): 로 코드 블록을 감싸서 autograd가 .requires_grad=True인 Tensor들의 연산 기록을 추적하는 것을 멈출 수 있습니다.

In [14]:
print(x.requires_grad)
print((x**2).requires_grad)

with torch.no_grad():
  print((x**2).requires_grad)

True
True
False
